In [1]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_categorias_autores=pd.read_csv("logCT2.csv",index_col=0)
df_categorias_autores['Category'].unique()

array(['Academic', 'Student', 'Professional'], dtype=object)

In [19]:
winners=['A1 2015',"A2 2015","A13 2017","A18 2017","A22 2017","A4 2019","A12 2019","A1 2020","A2 2020"]

In [20]:
df_categorias_autores["repYear"]=[rep+" "+str(year) for rep,year in zip(df_categorias_autores['Report'],df_categorias_autores['Year'])]

In [22]:
df_winners=df_categorias_autores[df_categorias_autores["repYear"].isin(winners)]

In [25]:
df_no_winners=df_categorias_autores[~df_categorias_autores["repYear"].isin(winners)]

In [27]:
df_no_winners.head()

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Type,repYear
4,A7 P2 BPI 2020,Calculate throughput,,2021-10-19 12:47:13.682150,A7,P2,2020,Academic,Analyze Cycle Time,A7 2020
5,A7 P2 BPI 2020,Comparative of throughput,,2021-10-19 12:47:13.682150,A7,P2,2020,Academic,Analyze Cycle Time,A7 2020
6,A7 P2 BPI 2020,Comparative of throughput,,2021-10-19 12:47:13.682150,A7,P2,2020,Academic,Analyze Cycle Time,A7 2020
7,A7 P2 BPI 2020,Represent scatterplot,,2021-10-19 12:47:13.697738,A7,P2,2020,Academic,Representation,A7 2020
8,A10 P2 BPI 2020,Represent boxplots of cycle time,,2021-10-19 12:47:13.697738,A10,P2,2020,Academic,Representation,A10 2020


In [30]:
intersection = len(list(set(df_winners["concept:name"]).intersection(df_no_winners["concept:name"])))
union = (len(set(df_winners["concept:name"])) + len(set(df_no_winners["concept:name"]))) - intersection
float(intersection) / union

0.42857142857142855

In [31]:
intersection = len(list(set(df_winners["Specific operation"]).intersection(df_no_winners["Specific operation"])))
union = (len(set(df_winners["Specific operation"])) + len(set(df_no_winners["Specific operation"]))) - intersection
float(intersection) / union

0.37142857142857144

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
#Intento de Jaccard 2:

In [64]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
    print("similitud: "+str(float(tamIntersection)/tamUnion))

In [65]:
jaccardIndex(df_winners["concept:name"],df_no_winners["concept:name"])

similitud: 0.42857142857142855


In [71]:
jaccardIndex(df_winners["Specific operation"],df_no_winners["Specific operation"])

similitud: 0.37142857142857144


In [67]:
#------------------------------------------------------------------------------------------------------------------------------
#ganadores vs no ganadores por categoria:

In [68]:
#Academico
df_winners_academic=df_winners[df_winners['Category']=="Academic"]
df_no_winners_academic=df_no_winners[df_no_winners['Category']=="Academic"]

In [75]:
#operaciones academic
jaccardIndex(df_winners_academic['concept:name'],df_no_winners_academic['concept:name'])

similitud: 0.4482758620689655


In [78]:
#variantes academic
jaccardIndex(df_winners_academic['Specific operation'],df_no_winners_academic['Specific operation'])

similitud: 0.29411764705882354


In [76]:
#operaciones student
df_winners_student=df_winners[df_winners['Category']=="Student"]
df_no_winners_student=df_no_winners[df_no_winners['Category']=="Student"]
jaccardIndex(df_winners_student['concept:name'],df_no_winners_student['concept:name'])

similitud: 0.35294117647058826


In [79]:
#variantes student
jaccardIndex(df_winners_student['Specific operation'],df_no_winners_student['Specific operation'])

similitud: 0.3050847457627119


In [77]:
#operaciones professional
df_winners_pro=df_winners[df_winners['Category']=="Professional"]
df_no_winners_pro=df_no_winners[df_no_winners['Category']=="Professional"]
jaccardIndex(df_winners_pro['concept:name'],df_no_winners_pro['concept:name'])

similitud: 0.20454545454545456


In [80]:
#variantes professional
jaccardIndex(df_winners_pro['Specific operation'],df_no_winners_pro['Specific operation'])

similitud: 0.10344827586206896


In [ ]:
#------------------------------------------------------------------------------------------------------------------------------

In [85]:
def calculaMediaOperacionesTraza(df):
    return len(df['concept:name'])/len(list(set(df['case:concept:name'])))

In [88]:
mediaWinners=calculaMediaOperacionesTraza(df_winners)
mediaNoWinners=calculaMediaOperacionesTraza(df_no_winners)
print("Promedio operaciones por traza ganadores: "+str(mediaWinners))
print("Promedio operaciones por traza no ganadores: "+str(mediaNoWinners))

Promedio operaciones por traza ganadores: 6.0
Promedio operaciones por traza no ganadores: 7.745454545454545


In [86]:
mediaAcademicosWinners=calculaMediaOperacionesTraza(df_winners_academic)
mediaAcademicosNoWinners=calculaMediaOperacionesTraza(df_no_winners_academic)
print("Promedio operaciones por traza ganadores academicos: "+str(mediaAcademicosWinners))
print("Promedio operaciones por traza no ganadores academicos: "+str(mediaAcademicosNoWinners))

Promedio operaciones por traza ganadores academicos: 7.166666666666667
Promedio operaciones por traza no ganadores academicos: 5.230769230769231


In [87]:
mediaStudentWinners=calculaMediaOperacionesTraza(df_winners_student)
mediaStudentNoWinners=calculaMediaOperacionesTraza(df_no_winners_student)
print("Promedio operaciones por traza ganadores estudiantes: "+str(mediaStudentWinners))
print("Promedio operaciones por traza no ganadores estudiantes: "+str(mediaStudentNoWinners))

Promedio operaciones por traza ganadores estudiantes: 4.25
Promedio operaciones por traza no ganadores estudiantes: 10.954545454545455


In [89]:
mediaProWinners=calculaMediaOperacionesTraza(df_winners_pro)
mediaProNoWinners=calculaMediaOperacionesTraza(df_no_winners_pro)
print("Promedio operaciones por traza ganadores profesionales: "+str(mediaProWinners))
print("Promedio operaciones por traza no ganadores profesionales: "+str(mediaProNoWinners))

Promedio operaciones por traza ganadores profesionales: 9.5
Promedio operaciones por traza no ganadores profesionales: 7.661290322580645


In [ ]:
#-------------------------------------------------------------------------------

In [94]:
def calculaMediaOperacionesTraza2(df):
    return len(df[df['Specific operation']!=" "])/len(list(set(df['case:concept:name'])))

In [96]:
mediaWinners=calculaMediaOperacionesTraza2(df_winners)
mediaNoWinners=calculaMediaOperacionesTraza2(df_no_winners)
print("Promedio variantes por traza ganadores: "+str(mediaWinners))
print("Promedio variantes por traza no ganadores: "+str(mediaNoWinners))

Promedio variantes por traza ganadores: 5.375
Promedio variantes por traza no ganadores: 6.372727272727273


In [97]:
mediaAcademicosWinners=calculaMediaOperacionesTraza2(df_winners_academic)
mediaAcademicosNoWinners=calculaMediaOperacionesTraza2(df_no_winners_academic)
print("Promedio variantes por traza ganadores academicos: "+str(mediaAcademicosWinners))
print("Promedio variantes por traza no ganadores academicos: "+str(mediaAcademicosNoWinners))

Promedio variantes por traza ganadores academicos: 6.166666666666667
Promedio variantes por traza no ganadores academicos: 4.346153846153846


In [98]:
mediaStudentWinners=calculaMediaOperacionesTraza2(df_winners_student)
mediaStudentNoWinners=calculaMediaOperacionesTraza2(df_no_winners_student)
print("Promedio variantes por traza ganadores estudiantes: "+str(mediaStudentWinners))
print("Promedio variantess por traza no ganadores estudiantes: "+str(mediaStudentNoWinners))

Promedio variantes por traza ganadores estudiantes: 3.875
Promedio variantess por traza no ganadores estudiantes: 9.136363636363637


In [99]:
mediaProWinners=calculaMediaOperacionesTraza2(df_winners_pro)
mediaProNoWinners=calculaMediaOperacionesTraza2(df_no_winners_pro)
print("Promedio variantes por traza ganadores profesionales: "+str(mediaProWinners))
print("Promedio variantes por traza no ganadores profesionales: "+str(mediaProNoWinners))

Promedio variantes por traza ganadores profesionales: 9.0
Promedio variantes por traza no ganadores profesionales: 6.241935483870968


In [52]:
#------------------------------------------------------------------------------------------------------------------------------
#intento ganadores codificado

In [46]:
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import LabelEncoder
df2=df_categorias_autores
label_encoder = LabelEncoder()
df2['concept:name'] = label_encoder.fit_transform(df2['concept:name'])

In [49]:
df_winners_2=df2[df2["repYear"].isin(winners)]
df_no_winners_2=df2[~df2["repYear"].isin(winners)]